In [1]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

import itertools

In [2]:
import warnings
warnings.simplefilter(action="ignore")

In [3]:
companies = ["AMAZON", "APPLE", "GOOGLE", "META", "NETFLIX"]
time_period = ["daily"]

In [4]:
for company, period in list(itertools.product(companies, time_period)):
    
    date_parse = lambda dates: pd.to_datetime(dates)
    path = f"/diploma_info/datalake/raw_data/{company}_{period}.csv"

    full_set = pd.read_csv(
        path,
        parse_dates=["Date"],
        date_parser=date_parse,
        index_col=["Date"],
    )


    full_set.index.name = 'date'
    full_set.columns = [c.lower() for c in full_set.columns]

    growth = [0]
    diff_value = [0]


    for k in range(1, full_set.shape[0]):
        diff_value.append(full_set.iloc[k]["close"] - full_set.iloc[k-1]["close"])
        if diff_value[-1] > 0:
            growth.append(1)
        else:
            growth.append(0)

    full_set['diff_value'] = diff_value
    full_set['growth'] = growth

    full_set['year'] = full_set.index.year
    full_set['month'] = full_set.index.month
    full_set['day'] = full_set.index.day
    full_set['day_of_week'] = full_set.index.weekday
    full_set['week_of_year'] = (full_set.index.isocalendar()['week']).astype('int')
    full_set['close_lag_1'] = full_set['close'].shift(1).bfill()
    full_set['volume_lag_1'] = full_set['volume'].shift(1).bfill()

    for window in [3, 5, 7]:
        close_agg = pd.DataFrame(round(full_set['close'].rolling(window=window, closed='left').agg(
            ('max', 'min', 'mean')
        )))
        close_agg.columns = [f'close_max_{window}_days', f'close_min_{window}_days', f'close_mean_{window}_days']
        day_mean = close_agg.reset_index()[['date', f'close_max_{window}_days', f'close_min_{window}_days', f'close_mean_{window}_days']]

        full_set = full_set.reset_index().merge(day_mean, on='date').set_index("date")
#         full_set = full_set.loc[:test_end.strftime("%Y%m%d"),]


    full_set = full_set.fillna(0)
     
    full_set.to_csv(f"/diploma_info/datalake/processed_data/{company}_{period}.csv")

In [5]:
full_set

,open,high,low,close,adj close,volume,diff_value,growth,year,month,...,volume_lag_1,close_max_3_days,close_min_3_days,close_mean_3_days,close_max_5_days,close_min_5_days,close_mean_5_days,close_max_7_days,close_min_7_days,close_mean_7_days
date,,,,,,,,,,,,,,,,,,,,,
2002-05-23,1.156429,1.242857,1.145714,1.196429,1.196429,104790000,0.000000,0,2002,5,...,104790000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2002-05-24,1.214286,1.225000,1.197143,1.210000,1.210000,11104800,0.013571,1,2002,5,...,104790000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2002-05-28,1.213571,1.232143,1.157143,1.157143,1.157143,6609400,-0.052857,0,2002,5,...,11104800.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2002-05-29,1.164286,1.164286,1.085714,1.103571,1.103571,6757800,-0.053572,0,2002,5,...,6609400.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2002-05-30,1.107857,1.107857,1.071429,1.071429,1.071429,10154200,-0.032142,0,2002,5,...,6757800.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-26,583.559998,590.750000,583.270020,587.650024,587.650024,2393700,4.090027,1,2024,2,...,2568900.0,588.0,573.0,582.0,588.0,573.0,581.0,593.0,573.0,582.0
2024-02-27,595.000000,605.359985,592.330017,601.669983,601.669983,4489400,14.019958,1,2024,2,...,2393700.0,588.0,584.0,587.0,588.0,573.0,582.0,593.0,573.0,584.0
2024-02-28,595.789978,598.169983,590.010010,596.479980,596.479980,2605200,-5.190002,0,2024,2,...,4489400.0,602.0,584.0,591.0,602.0,573.0,587.0,602.0,573.0,585.0
